# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey
# config gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the mouse data and the study results
weather_df = pd.read_csv('../output_data/cities.csv') 
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,andrews,90,US,1613008409,100,32.3187,-102.5457,26.60,13.80
1,1,mineiros,100,BR,1613008374,95,-17.5694,-52.5511,66.24,3.04
2,2,severo-kurilsk,100,RU,1613008239,91,50.6789,156.1250,28.00,19.55
3,3,mataura,100,NZ,1613008162,87,-46.1927,168.8643,50.25,13.06
4,4,lakselv,90,NO,1613008413,86,70.0513,24.9718,19.40,4.61
...,...,...,...,...,...,...,...,...,...,...
540,540,baiao,97,BR,1613009186,94,-2.7906,-49.6717,74.79,3.74
541,541,catalina,66,US,1613009187,30,32.5056,-110.9211,64.00,5.48
542,542,boyuibe,69,BO,1613009188,92,-20.4167,-63.2833,60.42,3.31
543,543,garoua,92,CM,1613009189,20,9.3000,13.4000,68.47,1.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
locations=weather_df[["Lat","Lng"]]
weight=weather_df.Humidity


545


In [25]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2) #set center/zoom so the map isn't huge and repeated on the page

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100
                                 ,point_radius=1)  # point radius how big shape around 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#narrow down weather cities df to cities that have the right vacation weather.  use 3 criteria to narrow cities down to ~10 choices
vacay_humidity_limit = 23
vacay_temp_max = 80
vacay_temp_min = 20
vacay_df = weather_df.loc[(weather_df.Humidity<vacay_humidity_limit)&(weather_df.Max_Temp<=vacay_temp_max)&(weather_df.Max_Temp>vacay_temp_min)]
#no need to drop null rows as that was done when building the weather df orifinally.
vacay_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
40,40,jega,0,NG,1613008465,20,12.2175,4.3792,71.26,8.08
62,62,silver city,90,US,1613008495,22,32.7701,-108.2803,51.80,6.91
100,100,kolondieba,0,ML,1613008546,20,11.0882,-6.8926,72.68,3.40
122,122,bilma,0,NE,1613008578,22,18.6853,12.9164,66.27,6.44
206,206,araouane,8,ML,1613008692,18,18.9048,-3.5265,68.22,10.42
296,296,dakoro,0,NE,1613008821,20,14.5106,6.7650,66.88,10.31
333,333,koulikoro,43,ML,1613008879,19,14.0000,-7.7500,73.11,8.19
363,363,ojinaga,1,MX,1613008921,13,29.5667,-104.4167,69.80,9.22
397,397,tessalit,1,ML,1613008929,20,20.1986,1.0114,65.79,9.53
427,427,beboto,100,TD,1613009011,21,8.2668,16.9390,68.38,3.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = vacay_df
hotel_df["Hotel Name"]=""


C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
params = {"keyword":"hotel",
          "radius": 5000,
          "type": "lodging",
          "key": gkey    
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in hotel_df.iterrows():
    if row["City_ID"]==206:
        break   #test break to just call api 1x
    lat = row["Lat"]
    lng = row["Lng"]
    #print(row)
    params["location"]= f"{lat},{lng}"
    hotel_resp= requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_resp["results"][0]["name"]
    except:
        print("Issue API with response for city: "+row['City'], end =". ")  #change end so it's not a new line
        try:
            print("API status: "+hotel_resp['status'])  #if the API returned no results, it'll be noted in status
        except:
            print("Unexpected API response.")   #if no message data found, print something to get a newline for next log statement
    #print(json.dumps(hotel_resp["results"], indent=4))
    #possible functionality upgrade for next iteration: add loop through response to check business_status == "OPERATIONAL"
    
hotel_df    

Issue with response for city bilma. API status: ZERO_RESULTS


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
40,40,jega,0,NG,1613008465,20,12.2175,4.3792,71.26,8.08,Jega Guest Inn
62,62,silver city,90,US,1613008495,22,32.7701,-108.2803,51.80,6.91,Quality Inn
100,100,kolondieba,0,ML,1613008546,20,11.0882,-6.8926,72.68,3.40,Hotel Dakan
122,122,bilma,0,NE,1613008578,22,18.6853,12.9164,66.27,6.44,
206,206,araouane,8,ML,1613008692,18,18.9048,-3.5265,68.22,10.42,
296,296,dakoro,0,NE,1613008821,20,14.5106,6.7650,66.88,10.31,
333,333,koulikoro,43,ML,1613008879,19,14.0000,-7.7500,73.11,8.19,
363,363,ojinaga,1,MX,1613008921,13,29.5667,-104.4167,69.80,9.22,
397,397,tessalit,1,ML,1613008929,20,20.1986,1.0114,65.79,9.53,
427,427,beboto,100,TD,1613009011,21,8.2668,16.9390,68.38,3.20,


In [15]:
hotel_resp

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']] 

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))